In [ ]:
suppressMessages(library(ggplot2))
suppressMessages(library(data.table))
options(repr.plot.width=14, repr.plot.height=7)

In [ ]:
if (getwd() != "/projects/ps-gymreklab/amassara/happler/analysis") {
    setwd("/projects/ps-gymreklab/amassara/happler/analysis")
}

In [ ]:
gt = data.table::fread("out/19_45401409-46401409/sim/hap/0.05/happler/merged_happler.tsv.gz", sep="\t", header=T, stringsAsFactors = FALSE, check.names=TRUE, data.table=FALSE)
finemap_results = readRDS("out/19_45401409-46401409/sim/hap/0.05/exclude/happler/finemap.rds")
susie_results = readRDS("out/19_45401409-46401409/sim/hap/0.05/exclude/happler/susie.rds")
hap = FALSE
if (nchar("out/19_45401409-46401409/sim/hap/19_45401409-46401409.hap") > 0) {
    hap = TRUE
    happler_haplotype = file("out/19_45401409-46401409/sim/hap/0.05/happler/happler.hap", "rt")
    causal_haplotype = file("out/19_45401409-46401409/sim/hap/19_45401409-46401409.hap", "rt")
}

In [ ]:
if (hap) {
    write("Parsing hap files", stderr())
    while(TRUE) {
        line = readLines(happler_haplotype, 1)
        if (grepl("^H", line)) break
    }
    # extract the start and end of the haplotype
    happler_hap = as.integer(strsplit(line, "\t")[[1]][c(3,4)])
    names(happler_hap) = c("start", "end")
    while(TRUE) {
        line = readLines(causal_haplotype, 1)
        if (grepl("^H", line)) break
    }
    # extract the start, end, and ID of the haplotype
    causal_hap = as.integer(strsplit(line, "\t")[[1]][c(3,4)])
    names(causal_hap) = c("start", "end")
    haplotypes = t(data.frame(causal_hap, happler_hap))
}

In [ ]:
X = as.matrix(gt[,-1])
storage.mode(X) = 'double'

In [ ]:
# parse the susie data:
# 1) the truly causal variant, as defined in the simulation
# 2) whether the causal variant was provided in the genotypes
# 3) the list provided by susie as output
causal_variant = susie_results$causal_var
exclude_causal = susie_results$causal_excluded
fitted = susie_results$fitted
susie_pip = fitted$pip

In [ ]:
# first, we must create a vector with the causal status
# this vector indicates which variant is truly causal
b = rep(0, ncol(X))
names(b) = colnames(X)
if (exclude_causal) {
    b = b[!(names(b) %in% c(causal_variant))]
    if (hap) {
        haplotypes = t(data.frame(haplotypes[c(2),]))
    }
} else {
    b[causal_variant] = 1
}

In [ ]:
# parse the finemap data
snp = finemap_results[[1]]$snp
finemap_pip = snp[order(as.numeric(snp$snp)),]$snp_prob

In [ ]:
# define a function that generates the PIP plot data
pip_plot_data = function(pips, X, b, susie_cs=NULL) {
    # note that each of pips, X, and b must be ordered by variant POS
    # first, initialize the values we need
    b_colors = c(`0`='black', `1`='red')
    causal_var = names(b[b == 1])
    if (length(causal_var) == 0) {
        causal_var = X[,causal_variant]
        X = X[,!(colnames(X) %in% c(causal_variant))]
    } else {
        causal_var = X[,causal_var[1]]
    }
    data = data.frame(
        pip = pips,
        b = as.character(b),
        pos = as.integer(sub("\\.(0|1|2)", "", sub("X(\\S+)", "\\1", names(b)))),
        ld_causal = as.vector(cor(causal_var, X))^2
    )
    if (!is.null(susie_cs)) {
        data$cs = as.integer((names(b) %in% susie_cs))*2
    } else {
        data$cs = 0
    }
    return(data)
}

In [ ]:
# define a function for PIP plotting
pip_plot = function(pips, X, b, susie_cs=NULL) {
    # create a ggplot of the PIPs
    # but first, get the data we need for the plot
    data = pip_plot_data(pips, X, b, susie_cs)
    # extract the causal variants to another data frame
    data_causal = data[data$b == '1',]
    # make the plot
    ggplot(data, aes(x=pos, y=pip)) +
    geom_point(aes(fill=ld_causal, stroke=cs, color=factor(cs)), size=7, shape=21) +
    scale_fill_gradient(name='LD with Causal Variant', low='#FBBA72', high='#691E06') +
    scale_color_manual(name='Credible Sets', values=c('transparent', '#7C9299'), guide="none") +
    geom_point(data=data_causal, aes(stroke=cs, color=factor(cs)), fill='red', size=7, shape=21) +
    xlab('Chromosomal Position') +
    ylab('Posterior Inclusion Probability (PIP)') + 
    ylim(0,1) +
    theme_grey(base_size=16)
}

In [ ]:
pip_plot(susie_pip, X, b, susie_cs=names(susie_pip[fitted$sets$cs[['L1']]]))

In [ ]:
# define a function for PIP plotting with haplotypes as lines
pip_plot_haps = function(pips, X, b, haplotypes, susie_cs=NULL) {
    # create a ggplot of the PIPs
    # but first, get the data we need for the plot
    data = pip_plot_data(pips, X, b, susie_cs)
    # extract the haplotypes to another data frame
    data_hap = cbind(data[grepl(".2", rownames(data), fixed=T),], haplotypes)
    data_hap$color = c("black", "red")[as.integer(data_hap$b)+1]
    # remove the haps from the data
    data = data[!(row.names(data) %in% rownames(data_hap)),]
    # make the plot
    ggplot(data, aes(x=pos, y=pip)) +
    geom_point(aes(fill=ld_causal, stroke=cs, color=factor(cs)), size=7, shape=21) +
    scale_fill_gradient(name='LD with Causal Variant', low='#FBBA72', high='#691E06') +
    scale_color_manual(name='Credible Sets', values=c('transparent', '#7C9299'), guide="none") +
    geom_segment(data=data_hap, aes(x = start, xend = end, y = pip, yend = pip, color=factor(b)), color=data_hap$color, size=8) +
    xlab('Chromosomal Position') +
    ylab('Posterior Inclusion Probability (PIP)') +
    ylim(0,1) +
    theme_grey(base_size=16)
}

In [ ]:
pip_plot_haps(susie_pip, X, b, haplotypes, susie_cs=names(susie_pip[fitted$sets$cs[['L1']]]))